In [1]:
import xlwings as xw
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

# settings
## to do: YAML these
## or config them in Excel

sample_size = 70 # initial sample to optimize on
number_of_samples = 10 # not used yet

# wb = xw.Book()  # this will create a new workbook
wb = xw.Book(r'myproject/myproject.xlsm')  # connect to an existing file in the current working directory
# wb = xw.Book(r'C:\path\to\file.xlsx')  # on Windows: use raw strings to escape backslashes

Read in product and channel information from Excel.

In [2]:
customer_sheet = wb.sheets['customer_data']
product_sheet = wb.sheets['products']
channel_sheet = wb.sheets['channels']
# scenario_sheet = wb.sheets['Scenario']

In [3]:
product_probs_all = customer_sheet.range('A1').options(pd.DataFrame, expand='table').value
products_df = product_sheet.range('A1').options(pd.DataFrame, expand='table').value

products = products_df.index
productValue = products_df.iloc[:,0]

channels_df = channel_sheet.range('A1').options(pd.DataFrame, expand='table').value
channels = channels_df.index
cost = channels_df['cost']
factor = channels_df['factor']

sample_scaling = sample_size/product_probs_all.shape[0]

Get the available marketing budget from the `Scenario` sheet.

In [4]:
budget_range = scenario_sheet.range('budgetConstraints').value
availableBudget_total = budget_range[1]
availableBudget = availableBudget_total*sample_scaling # scale to sample_size for initial optimization
print("Sampled available budget: %d" % availableBudget)

Sampled available budget: 1296


Create a sample of size `sample_size` for the initial optimization.

> **To do**: adjust this to sparse notation to generalise better.

In [5]:
product_probs = product_probs_all.sample(n=sample_size, random_state=2058)

Instantiate the solver as an MIP problem.

In [6]:
solver = pywraplp.Solver('SolveCampaignProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver.Clear()

Define the number of customers, the number of offers and the number of channels as $x_{ijk}$.

In [7]:
num_customers = product_probs.shape[0]
num_products = len(products)
num_channels = len(channels)

x = {}

for i in range(num_customers):
    for j in range(num_products):
        for k in range(num_channels):
            x[i, j, k] = solver.IntVar(0, 1, 'x[%i,%i,%i]' % (i, j, k))

print('Number of customers: %d' % num_customers)
print('Number of products: %d' % num_products)
print('Number of channels: %d' % num_channels)

Number of customers: 70
Number of products: 4
Number of channels: 3


## Set up the constraints

  1. Offer only one product per customer. _(**TO DO:** update this.)_
  2. Adhere to budget, channel and product constraints from the Excel spreadsheet.
  3. Adhere to number of offer constraints
  


In [8]:
    ## offer only one product per customer
    for i in range(num_customers):
        solver.Add(solver.Sum([x[i, j, k] 
                               for j in range(num_products)
                               for k in range(num_channels)
                              ]) <= 1) # *** MAGIC NUMBER ALERT!!! ***

    ## Do not exceed the budget
    solver.Add(solver.Sum([x[i, j, k]*cost[k]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)
                          ]) <= availableBudget)

### Get the channel constraints

Adjust the constraints for the sample size.

In [10]:
channels_df['minimum offers adjusted'] = channels_df['minimum offers']*sample_scaling
channels_df['maximum offers adjusted'] = channels_df['maximum offers']*sample_scaling
channels_df['minimum expenditure adjusted'] = channels_df['minimum expenditure']*sample_scaling
channels_df['maximum expenditure adjusted'] = channels_df['maximum expenditure']*sample_scaling
channels_df['minimum revenue adjusted'] = channels_df['minimum revenue']*sample_scaling
channels_df['maximum revenue adjusted'] = channels_df['maximum revenue']*sample_scaling


### Set the channel constraints

In [11]:
# minimums for channel
if channels_df['minimum offers adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'minimum offers adjusted']):
            solver.Add(solver.Sum([x[i, j, k]
                for i in range(num_customers)
                for j in range(num_products)
                ]) >= channels_df.loc[channels[k], 'minimum offers adjusted'])

# maxima for channel
if channels_df['maximum offers adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'maximum offers adjusted']):
            solver.Add(solver.Sum([x[i, j, k]
                for i in range(num_customers)
                for j in range(num_products)
                ]) <= channels_df.loc[channels[k], 'maximum offers adjusted'])

# minimums for channel
if channels_df['minimum expenditure adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'minimum expenditure adjusted']):
            solver.Add(solver.Sum([x[i, j, k]*cost[k]
                for i in range(num_customers)
                for j in range(num_products)
                ]) >= channels_df.loc[channels[k], 'minimum expenditure adjusted'])

# maximums for channel
if channels_df['maximum expenditure adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'maximum expenditure adjusted']):
            solver.Add(solver.Sum([x[i, j, k]*cost[k]
                for i in range(num_customers)
                for j in range(num_products)
                ]) <= channels_df.loc[channels[k], 'maximum expenditure adjusted'])

# minimums for channel
if channels_df['minimum revenue adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'minimum revenue adjusted']):
            print('Minimum revenue %d for %s' % (channels_df.loc[channels[k], 'minimum revenue adjusted'], 
                                                channels[k]))
            solver.Add(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                for i in range(num_customers)
                for j in range(num_products)
                ]) >= channels_df.loc[channels[k], 'minimum revenue adjusted'])

# maximums for channel
if channels_df['maximum revenue adjusted'].notnull().any():
    for k in range(num_channels):
        if pd.notnull(channels_df.loc[channels[k], 'maximum revenue adjusted']):
            print('Maximum revenue %d (%d) for %s' % (channels_df.loc[channels[k], 'maximum revenue adjusted'], 
                                                      channels_df.loc[channels[k], 'maximum revenue'],
                                                channels[k]))
            solver.Add(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                for i in range(num_customers)
                for j in range(num_products)
                ]) <= channels_df.loc[channels[k], 'maximum revenue adjusted'])

Maximum revenue 2100 (300000) for seminar


### Get the product constraints

Adjust the constraints for the sample size.

In [12]:
products_df['minimum offers adjusted'] = products_df['minimum offers']*sample_scaling
products_df['maximum offers adjusted'] = products_df['maximum offers']*sample_scaling
products_df['minimum expenditure adjusted'] = products_df['minimum expenditure']*sample_scaling
products_df['maximum expenditure adjusted'] = products_df['maximum expenditure']*sample_scaling
products_df['minimum revenue adjusted'] = products_df['minimum revenue']*sample_scaling
products_df['maximum revenue adjusted'] = products_df['maximum revenue']*sample_scaling


### Set the product constraints

In [13]:
# minima for product
if products_df['minimum offers adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'minimum offers adjusted']):
            solver.Add(solver.Sum([x[i, j, k]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) >= products_df.loc[products[j], 'minimum offers adjusted'])

# maxima for product
if products_df['maximum offers adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'maximum offers adjusted']):
            solver.Add(solver.Sum([x[i, j, k]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) <= products_df.loc[products[j], 'maximum offers adjusted'])

# minima for product
if products_df['minimum expenditure adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'minimum expenditure adjusted']):
            solver.Add(solver.Sum([x[i, j, k]*cost[k]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) >= products_df.loc[products[j], 'minimum expenditure adjusted'])

# maxima for product
if products_df['maximum expenditure adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'maximum expenditure adjusted']):
            solver.Add(solver.Sum([x[i, j, k]*cost[k]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) <= products_df.loc[products[j], 'maximum expenditure adjusted'])

# Is this causing the infeasible solution?

# minima for product
if products_df['minimum revenue adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'minimum revenue adjusted']):
            print('Minimum revenue %d (%d) for %s' % (products_df.loc[products[j], 'minimum revenue adjusted'], 
                                                      products_df.loc[products[j], 'minimum revenue'],
                                                products[j]))

            solver.Add(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) >= products_df.loc[products[j], 'minimum revenue adjusted'])

# maxima for product
if products_df['maximum revenue adjusted'].notnull().any():
    for j in range(num_products):
        if pd.notnull(products_df.loc[products[j], 'maximum revenue adjusted']):
            print('Maximum revenue %d (%d) for %s.' % (products_df.loc[products[j], 'maximum revenue adjusted'], 
                                                      products_df.loc[products[j], 'maximum revenue'],
                                                products[j]))
            solver.Add(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                for i in range(num_customers)
                for k in range(num_channels)
                ]) <= products_df.loc[products[j], 'maximum revenue adjusted'])

Maximum revenue 1400 (200000) for Pension.


## Set the _objective function_

Set to maximise the revenue $R$. Here $x_{ijk}$ denotes whether customer $i$ receives an offer for product $j$ over channel $k$, $f_k$ denotes the channel adjustment factor, $v_j$ the product value and $p_{ij}$ the probability that customer $i$ takes up product $j$.

$ \max R = \sum_{ijk} x_{ijk} \times f_k \times v_j \times p_{ij}$


> At some point, need to be able to specify 
  1. What to optimize, and 
  2. Whether to maximise or minimise.  

> At the moment we maximise revenue, this could be profit, we could minimise budget, maximise profit or ~~maximise ROI.~~

In [14]:
#    solver.Minimize(solver.Sum([cost[i][j] * x[i, j] for i in range(num_workers)
#                                                     for j in range(num_tasks)]))
# optimize = 'Profit' # to do: get this from the interface
optimize = 'Expenditure'

if optimize == 'Revenue':
    solver.Maximize(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                               for i in range(num_customers)
                               for j in range(num_products)
                               for k in range(num_channels)]))
elif optimize == 'Profit':
    solver.Maximize(solver.Sum([x[i, j, k]*(factor[k]*productValue[j]*product_probs[products[j]].iloc[i] - cost[k])
                               for i in range(num_customers)
                               for j in range(num_products)
                               for k in range(num_channels)]))
elif optimize == 'Expenditure':
        solver.Minimize(solver.Sum([x[i, j, k]*cost[k]
                               for i in range(num_customers)
                               for j in range(num_products)
                               for k in range(num_channels)]))


### Invoke the solver

> Need a routine here to evaluate whether the solver is solving. That is, set the most iterations and a time limit.

In [15]:
# Invoke the solver
# t = time.process_time()
sol = solver.Solve()
# elapsed_time = time.process_time() - t
print('Solver completed with return value %d.' % sol)

Solver completed with return value 0.


> **To do:** If not returned 0, throw an error.

I guess `sol == 0` means that the solver correctly solved. Values of $1$ or $2$ mean something else.

Print out the solution. We can print out more information about the constraints. What happens in `xlwings` when the python routine prints – does it go to the logs?

In [16]:
report = [(channels[k], products[j], product_probs.name.iloc[i], x[i, j, k].solution_value()*cost[k],
          x[i, j, k].solution_value()*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]) 
          for i in range(num_customers) 
          for j in range(num_products) 
          for k in range(num_channels)  
           if x[i, j, k].solution_value() > 0 # else 0
         ]

report_bd = pd.DataFrame(report, columns=['channel', 'product', 'customer', 'cost', 'revenue'])

print('Total revenue = %d' % (solver.Objective().Value()))
print('Total budget  = %d' % (report_bd['cost'].sum()) )



# display(report_bd)

Total revenue = 0
Total budget  = 0


Channel counts.

In [17]:
report_bd.groupby(['channel', 'product']).count()

,,customer,cost,revenue
channel,product,,,


The sample has given us the rough outline of the optimization. Using these figures, replicate using non-linear minimization.

In [ ]:
n_obs_orig = num_customers
n_obs_new = product_probs_all.shape[0]

In [ ]:
product_probs = product_probs_all

n_obs = product_probs.shape[0]

adjustment_factor = n_obs/n_obs_orig
availableBudget = availableBudget_total

# product_probs.head()

In [ ]:
num_customers = product_probs.shape[0]

offer_scale = int(n_obs_new/n_obs_orig)

# get the offers from the original optimization by product and channel
sample_counts = pd.pivot_table(report_bd, index='channel', columns='product', values='customer', 
                                   aggfunc=len, fill_value=0)

offers = sample_counts.stack()*offer_scale

The variable `offers` has a MultiIndex. We want this for the `product_profit` data frame. We can construct it from `channels` and `products`.

In [ ]:
# offers_ndx = pd.MultiIndex.from_product([channels, products], names=['channel', 'product'])
product_profit = pd.DataFrame(index=product_probs.index, columns=offers.index)

In [ ]:
for ch in offers.index.get_level_values('channel').unique():
    for pr in offers.index.get_level_values('product').unique():
        product_profit.loc[:, (ch, pr)] = product_probs[pr]*productValue[pr]
        product_profit.loc[:, (ch, pr)] = product_profit.loc[:, (ch, pr)]*factor[ch]
        

# The world of R

As of yet, the non-linear minimization in Python has not worked properly, but it _has_ with R and `nlm()`. Until I can get it to work, the workaround is to use `rpy2` to run R from Python.

> **To do:** Get the non-linear minimization right in Python.

Import the requisite libraries.

In [ ]:
import rpy2.robjects as robjects

In [ ]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')
stats = importr('stats')
data_table = importr('data.table')

In [ ]:
# select a mirror for R packages
# utils.chooseCRANmirror(ind=1) # select the first mirror in the list

Install packages using R's `install.package`. (I should not have to do this again.)

In [ ]:
# R package names
# packnames = ('magrittr', 'dplyr', 'data.table', 'dtplyr', 'stringr')

# R vector of strings
# from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
# for x in packnames:
#    if not(rpackages.isinstalled(x)):
#        utils.install_packages(StrVector(names_to_install))

All I need to run in R is the non-linear minimization, and whatever is needed to supply the appropiate data. Here is the original R code.

### The dual function (R)

```
            dual <- function(u, pp, offers) {
              if (dim(pp)[2] != length(u)) {
                print(c(dim(pp)[2], length(u)))
                stop("Mismatched dimensions")
                }
              d <- sweep(pp, 2, u)
              v <- apply(d, 1, max) 
              v[v < 0] <- 0
              y <- offers%*%u + sum(v)
              y
            }
```

### The optimisation (R)

```{r}
u_init <- offers*0
out <- nlm(dual, p=u_init, pp=product_profit, offers=offers, print.level = 1)
```

### Getting the solution (R)

```{r}
mindual <- out$minimum
u <- out$estimate
mindual
u
```

In [ ]:
robjects.r('''
        # create a function `dual`
            dual <- function(u, pp, offers) {
              if (dim(pp)[2] != length(u)) {
                print(c(dim(pp)[2], length(u)))
                stop("Mismatched dimensions")
                }
              d <- sweep(pp, 2, u)
              v <- apply(d, 1, max) 
              v[v < 0] <- 0
              y <- offers%*%u + sum(v)
              y
            }
        ''')

### Test the new function

To do this, need to create the `product_profit` array in R.

Import the required libraries and activate the interface between R and `pandas`.

In [ ]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [ ]:
pp_columns = product_profit.columns # if I need them

In [ ]:
product_profit.to_csv('temp_product_profit.csv', index=False, header=False)

In [ ]:
r_product_profit = data_table.fread('temp_product_profit.csv')

In [ ]:
# u_test = robjects.FloatVector([11.2, 15, 6.02, 19.5, 0, 4.98, 2.23, 7.75, 50.2, 23.2, 9.09, 35.2])
r_offers = robjects.IntVector(offers)

~~Test the dual function.~~

In [ ]:
dual = robjects.r['dual']
# dual(u_test, pp=r_product_profit, offers=r_offers)

Perform the non-linear minimisation.

In [ ]:
u_init = robjects.FloatVector(offers*0)
r_out = stats.nlm(dual, p=u_init, pp=r_product_profit, offers=r_offers, print_level=1)

Extract the estimate of $u$.

In [ ]:
r_u = r_out.rx('estimate')[0]

u = [r_u[i] if abs(r_u[i]) > 1e-5 else 0 for i in range(len(r_u))] # ugly way to convert

d = product_profit.sub(u) 
v = d.max(axis = 1)
v[v<0] = 0
ndx = np.argsort(-v)
d['customerid'] = d.index

## Allocate the optimised solution to customers

In [ ]:
d_melt = pd.melt(d, id_vars=['customerid']).sort_values(by=['customerid', 'value'], ascending=[True, False])

# Delete the offers from `d_melt` where already completely allocated.

allocated_counts = offers*0

offers_include = offers[allocated_counts < offers]
offers_include_df = pd.DataFrame(offers_include)
offers_include_df.reset_index(inplace=True)

d_melt = d_melt.merge(offers_include_df[['channel', 'product']], on=['channel', 'product'])

Create the initial allocation using the maximum value in each group. Will need to update `d_melt` once the first offer has been fully allocated.

In [ ]:
allocated_counts = offers*0


# d_melt.groupby(['variable']).agg({'value':'first'}).head()

# d_alloc = d_melt.groupby(['customerid']).first().sort_values(by=['value'], ascending=False)
d_alloc = d_melt.sort_values('value', ascending=False).drop_duplicates('customerid')

alloc_list = []


Repeat the next part until every offer in `offers` is allocated.

In [ ]:
counter = 0 # not sure where this goes yet
old_counter = counter
failsafe_threshold = 20000
failsafe = 0

while any(allocated_counts < offers): # could be no more offers to allocate
    offers_to_alloc = (allocated_counts < offers)
    
    # allocate while we haven't hit the limit for one of the offers
    while all((allocated_counts < offers) == (offers_to_alloc)):
        selected_offer = d_alloc.iloc[counter - old_counter]
        allocated_counts.loc[(selected_offer['channel'], selected_offer['product'])] += 1
        counter += 1
#     print(allocated_counts)      
    # note: selected_offer will contain the offer that has just been completely allocated!!
    print(selected_offer)
    
    ## allocate the selected offers
    
    print(counter, old_counter)
    d_alloc_select = d_alloc.iloc[[i for i in range(counter - old_counter)]]
    alloc_list.append(d_alloc_select)
    old_counter = counter
    
    ## delete the selected records from d_melt
    rows_to_keep = np.invert(d_melt.customerid.isin(d_alloc_select.customerid))
    d_melt = d_melt[rows_to_keep]
    
    ## delete the selected offers from the data frame
    offers_include = offers[allocated_counts < offers]
    offers_include_df = pd.DataFrame(offers_include)
    offers_include_df.reset_index(inplace=True)

    d_melt = d_melt.merge(offers_include_df[['channel', 'product']], on=['channel', 'product'])
    
    d_alloc = d_melt.sort_values('value', ascending=False).drop_duplicates('customerid')
    
    ## create the allocation data frame
    
    failsafe += 1
    if failsafe > failsafe_threshold:
        break # to protect against logic errors causing an infinite loop
    


Concatenate the allocation files to create the final allocation.

In [ ]:
final_allocation = pd.concat(alloc_list)
final_allocation = final_allocation.reset_index(drop=True)

## Calculate the profit and cost

In [ ]:
product_profit_allocated = pd.merge(pd.melt(product_profit.reset_index(), id_vars='customerid'), 
         final_allocation.drop('value', axis=1), on=['customerid', 'channel', 'product'], how="inner")

In [ ]:
channel_costs = pd.DataFrame(cost).reset_index()


In [ ]:
product_profit_allocated = pd.merge(product_profit_allocated, channel_costs, on='channel', how='left')


In [ ]:
def my_agg(df):
    names = {
        'offers':  df['value'].count(),
        'revenue': df['value'].sum(),
        'expenditure': df['cost'].sum()
    }
    return pd.Series(names, index=['offers', 'revenue', 'expenditure'])

In [ ]:
def summarize_benefits(df, grouper=None):
    if grouper == None:
        df['Total'] = 'Total'
        grouper = 'Total'
    df_grouped = df.groupby(grouper).apply(my_agg) 
    df_grouped['ROI'] = df_grouped['revenue']/df_grouped['expenditure']
#     df_grouped['investigations closed'].apply(lambda x: x if x > 0 else 1)

#     inv_formats = {
#         'offers': '{:,.0f}',
#         'revenue': '${:,.0f}',
#         'expenditure': '${:,.0f}',
#         'ROI': '{:.1%}'
#     }

#     return df_grouped.style.format(inv_formats)
    return df_grouped

## There should be a better way of doing this, as I am referencing each of these multiple times.

In [ ]:
ppa_total = summarize_benefits(product_profit_allocated)
ppa_channel = summarize_benefits(product_profit_allocated, 'channel')
ppa_product = summarize_benefits(product_profit_allocated, 'product')

Write to sheet `data_python`.

In [ ]:
data_python_sheet = wb.sheets['data_total']
data_channel_sheet = wb.sheets['data_channel']
data_product_sheet = wb.sheets['data_product']

In [ ]:
data_python_sheet.range('A1').value = ppa_total
data_channel_sheet.range('A1').value = ppa_channel
data_product_sheet.range('A1').value = ppa_product

## The end (for now)

> **To do**: write customer file in Excel or to file if large.

In [ ]:
pp_agg = product_profit_allocated.agg(aggregation)
pp_agg

In [ ]:
pp_channel = product_profit_allocated.groupby('channel').agg(aggregation)
pp_channel.columns = pp_channel.columns.droplevel(level=0)
pp_channel.rename(columns={"sum":"revenue", "sum":"expenditure", "count":"offers"})
pp_channel
# pp_agg.rename(columns={"min": "min_duration", "max": "max_duration", "mean": "mean_duration"})


Write to the named range.

In [ ]:
sht.range("test_value").value = test_value + 1
sht.range("test_value").value

In [ ]:
sht.range('A1').value = [['Foo 1', 'Foo 2', 'Foo 3'], [10.0, 20.0, 30.0]]
sht.range('A1').expand().value

**Powerful converters** handle most data types of interest, including Numpy arrays and Pandas DataFrames in both directions:

In [ ]:
import pandas as pd
df = pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])
sht.range('A1').value = df
sht.range('A1').options(pd.DataFrame, expand='table').value

**Matplotlib figures** can be shown as pictures in Excel:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot([1, 2, 3, 4, 5])
sht.pictures.add(fig, name='MyPlot', update=True)

Shortcut for the active sheet: `xw.Range`

If you want to quickly talk to the active sheet in the active workbook, you don’t need instantiate a workbook and sheet object, but can simply do:

In [ ]:
xw.Range('A1').value = 'Foo'
xw.Range('A1').value
'Foo'

## 2. Macros: Call Python from Excel

